## Text Network

### Install dependencies

In [3]:
import re
import itertools
import spacy
from nltk import FreqDist, bigrams
import numpy as np
import pandas as pd
import networkx as nx
from pyvis.network import Network
import seaborn as sns
from collections import Counter
sns.set(rc={'figure.figsize':(15,7)})


from nltk.tokenize.toktok import ToktokTokenizer
from nltk.corpus import stopwords
from youtube_transcript_api import YouTubeTranscriptApi
from gensim.models import KeyedVectors

In [117]:
def generate_transcript(id):
	transcript = YouTubeTranscriptApi.get_transcript(id,languages=['en'])
	script = ""

	for text in transcript:
		t = text["text"]
		if t != '[Music]':
			script += t + " "
		
	return script, len(script.split())

iden = 'zjK7PWmRRyg'

#ID é a parte final do link, depois do "=", exemplo 
#https://www.youtube.com/watch?v=L_jWHffIx5E
#L_jWHffIx5E

transcript, no_of_words = generate_transcript(iden)
print(transcript,no_of_words)

Hi I’m John Green and this is Crash Course
European History. So we’re going to turn our attention now
to the Industrial Revolution, one of the most significant developments in human history. Like, imagine with me that it’s 1820. I got this idea from the economist Robert
Gordon by the way. You live in, say, England. You probably work in agriculture. When you walk to town, you’re either pulling
your own cart, or if you’re lucky you have a horse. You have no running water or electricity. When you wash your few items of clothing,
you do so by hand. You cook over a fire. You think of time not primarily in minutes
and hours, but mostly in relationship to solar cycles--how close it is to night, or to morning,
or to midwinter. And in all these respects, your life in 1820
is basically identical to the lives of people in 1720, or 1520, or for that matter 1220. That’s not to say life hasn’t changed
in those hundreds of years--as we’ve explored in this series, lots has changed--but as Gregory
Clar

In [118]:
transcript2 = YouTubeTranscriptApi.get_transcript(iden,languages=['en'])

# dicionário do momento de cada fala
legend_dict = {}
for text in transcript2:
    legend_dict[text['start']] = text['text']
    


In [25]:

modelo = KeyedVectors.load_word2vec_format("C:/Users/rafae/Downloads/MindTube-main (2)/MindTube-main/notebook/wiki-news-300d-1M.vec")

KeyboardInterrupt: 

### Auxiliary functions

In [119]:
toktok = ToktokTokenizer()
stpwrd = stopwords.words('english')
stop_list = (['uh', 'also', 'oh', 'um', 'yeah', 'use', 'lot', 'put', 'get', 'would', 'gonna', 'really', 'much', 'actually', 'another'])
stpwrd.extend(stop_list)

'''for word in stpwrd:
    print(word)
    '''
    
    
#Acha as N palavras mais similares a string passada e retorna uma lista com as palavras. (Sem a métrica de proximidade)
def getSimilar(string, N):
    similarList = modelo.most_similar(positive=[string])[0:N]
    wordList = []
    for word in similarList:
        wordList.append(word)
    return wordList

def sort_list(list1, list2):
     
    list3 = []
    for entry in list2:
        list3.append(entry[1])
    zipped_pairs = zip(list3, list1)
 
    z = [x for _, x in sorted(zipped_pairs, reverse = True)]
 
    return z



#Busca na label dos nós por palavras similares a stopwords
def banStopword(lista):
    SUSwords = []
    similarWords = []
    for word in lista:
        similares = getSimilar(word,5)
        mostSimilar = 0
        similarWord = ''
        SUSWord = ''
        for palavra in similares:
            sentinel = 0
            if palavra[0] in stpwrd and palavra[1] > mostSimilar:
                mostSimilar = palavra[1]
                similarWord = palavra
                SUSWord = word
                sentinel = 1
                print(mostSimilar, similarWord, SUSWord)
            if sentinel == 1:  
                SUSwords.append(word)
                similarWords.append(palavra)

    return SUSwords, similarWords

'''word_tokens = toktok.tokenize(example_sent)

filtered_sentence = [w for w in word_tokens if not w.lower() in stop_words]

filtered_sentence = []

for w in word_tokens:
	if w not in stop_words:
		filtered_sentence.append(w)

print(word_tokens)
print(filtered_sentence)'''


def filtered_sentence(example_sent):
    word_tokens = toktok.tokenize(example_sent)
    filtered_sentence = [w for w in word_tokens if not w.lower() in stpwrd]
    return " ".join(filtered_sentence)

def remove_stopwords(text):
    doc = filtered_sentence(text)
    print(" ".join([token.text for token in doc if token.is_stop == False]))
    return " ".join([token.text for token in doc if token.is_stop == False])

def pre_processing(text):
    text = filtered_sentence(text)
    text = re.sub(r'[^\w\s]', '', text) #remove punctuation
    text = re.sub(r"\s{2,}", " ", text)
    return text

def co_occurrence(text):
    corpus = list(itertools.chain.from_iterable([text.split(' ')]))
    vocab = list(set(corpus))
    vocab_index = {word: i for i, word in enumerate(vocab)}
    bi_grams = list(bigrams(corpus))
    bigram_freq = FreqDist(bi_grams).most_common(len(bi_grams)) 
    co_occurrence_matrix = np.zeros((len(vocab), len(vocab))) 
    
    #bigram = ((word n, word n+1), num_occurrences)
    for bigram in bigram_freq:
        current = bigram[0][1] #word n+1
        previous = bigram[0][0] #word n
        count = bigram[1] #num_occurrences
        pos_current = vocab_index[current] #obtain id for word n+1 
        pos_previous = vocab_index[previous] #obtain id for word n 
        co_occurrence_matrix[pos_current][pos_previous] = count
        
    return co_occurrence_matrix, vocab_index

def structure_text_network(matrix, vocab_index):
    data_matrix = pd.DataFrame(matrix, index=vocab_index, columns=vocab_index)
    data_stack = data_matrix.stack()
    structure = data_stack[data_stack >= 1].rename_axis(('source', 'target')).reset_index(name='weight')
    return structure[(structure.source != " ") & (structure.target != " ")]

In [120]:
lista = ['one', 'measurements', 'what', 'also', 'me', 'mice', 'five', 'standard', 'stat', 'imagine', 'spread', 'weighed', 'look', 'mean', 'another', 'quest', 'quantifies', 'often', 'let', 'plot', 'set', 'deviation', 'data', 'measured', 'differences']

lista1, lista2 = banStopword(lista)

0.7335886359214783 ('another', 0.7335886359214783) one
0.6885168552398682 ('how', 0.6885168552398682) what
0.7364087700843811 ('a', 0.7364087700843811) another


In [121]:
print(lista1)
print('\n',lista2)
sort_list(lista1, lista2)

['one', 'what', 'another']

 [('another', 0.7335886359214783), ('how', 0.6885168552398682), ('a', 0.7364087700843811)]


['another', 'one', 'what']

In [122]:
getSimilar('Crowning',25)

[('crowning', 0.6477673053741455),
 ('Grandest', 0.5972409844398499),
 ('Crowned', 0.5812886953353882),
 ('Rejoicing', 0.5547146797180176),
 ('Surmounting', 0.5538868308067322),
 ('Boldest', 0.5468339323997498),
 ('Adorned', 0.5416989326477051),
 ('Symbolizes', 0.5404633283615112),
 ('Enthronement', 0.5369417071342468),
 ('Timelessness', 0.5364014506340027)]

### Generate text network

In [123]:
from spacy.lang.pt.stop_words import STOP_WORDS
nlp = spacy.load("pt_core_news_sm")
#STOP_WORDS.add('gotta')

In [124]:
text = transcript

In [154]:
text_cleaned = pre_processing(text)
matrix, vocab = co_occurrence(text_cleaned)
structure_network = structure_text_network(matrix, vocab)

v_title = "The Industrial Revolution: Crash Course European History #24"

title_words = pre_processing(v_title).split()
title_words

['Industrial', 'Revolution', 'Crash', 'Course', 'European', 'History', '24']

In [126]:
structure_network

,source,target,weight
0,greater,needed,1.0
1,machine,caught,1.0
2,machine,spinning,1.0
3,machine,jenny,1.0
4,machine,textile,1.0
...,...,...,...
1442,still,8,1.0
1443,still,created,1.0
1444,discouraged,norm,1.0
1445,discouraged,women,1.0


In [127]:
text_network = nx.DiGraph()
text_network = nx.from_pandas_edgelist(structure_network, 'target', 'source', ['weight'])

### PageRank

In [203]:
text_network = nx.DiGraph()
text_network = nx.from_pandas_edgelist(structure_network, 'target', 'source', ['weight'])
for i in text_network.nodes:
    for j in text_network.nodes:
        if i != j and i.lower() == j.lower():
            text_network = nx.contracted_nodes(text_network,i, j)

In [204]:
def pagerank_centrality(G):
    centrality =  nx.pagerank(G)
    return sorted(centrality.items(), key=lambda x: x[1], reverse=False)

In [205]:
centrality = pagerank_centrality(text_network)

def mean_simi(word,title):
    m = 0
    for i in title:
        try:
            m += modelo.similarity(word, i)
        except KeyError:
            m += 0
    return m/len(title)

centrality = [(x[0],x[1]*mean_simi(x[0],title_words)) for x in centrality]
#dicionário da centralidade
cen_dict = {}
max_cen = 0
for x in centrality:
    cen_dict[x[0]] = x[1]
    if max_cen < x[1]:
        max_cen = x[1]

### Graph Visualization

In [206]:
# lista dos nós
name_lst = [x[0] for x in centrality]

# lista de nós a serem removidos, estamos mantendo os 25 maiores
rmv_lst = name_lst[:-25]

# exibição do nome dos nós para a visualização
for i in text_network.nodes:
    text_network.nodes[i]['label'] = i

# redução do grafo mantendo a conectividade
for i in rmv_lst:
    maxi = 0
    for j in text_network.neighbors(i):
        w = name_lst.index(j)
        if maxi < w:
            maxi = w
            merge_node = j
    text_network.remove_node(i)

In [207]:
# momentos em que cada palavra é dita
dict_moments = {}
episilon = 45

for i in text_network.nodes:
    dict_moments[i] = []
    
for i in list(legend_dict.keys()):
    phrase = pre_processing(legend_dict[i])
    
    for j in text_network.nodes:
        if j in phrase.split() and (len(dict_moments[j]) == 0 or  dict_moments[j][-1]+episilon<i):
            dict_moments[j].append((i>1)*(int(i)-1))
            print(f"{j} was appended at {i}")
    
dict_moments['trade']        

Industrial was appended at 2.759
Like was appended at 10.65
work was appended at 18.91
water was appended at 26.4
time was appended at 34.219
people was appended at 44.989
life was appended at 44.989
work was appended at 78.29
time was appended at 87.6
people was appended at 90.82
even was appended at 95.75
see was appended at 102.75
could was appended at 112.7
Industrial was appended at 120.55
people was appended at 138.849
workers was appended at 152.68
work was appended at 158.04
century was appended at 161.4
water was appended at 174.159
people was appended at 194.91
time was appended at 194.91
Many was appended at 201.219
spinning was appended at 215.529
working was appended at 223.64
women was appended at 223.64
power was appended at 228.03
water was appended at 244.239
time was appended at 244.239
could was appended at 251.879
century was appended at 285.02
porcelain was appended at 301.81
make was appended at 307.09
could was appended at 307.09
people was appended at 310.52
eve

KeyError: 'trade'

In [208]:
nt = Network('800px', '1000px', bgcolor='#222222', notebook=True)
nt.from_nx(text_network)
nt.force_atlas_2based(gravity = -200)
for edg in nt.edges:
    # espessura e física das arestas
    #edg['width'] = text_network[i][j]['weight']**4
    #if edg['weight'] < 1:
    
    if edg['from'] == edg['to']:
        edg['hidden'] = True
    edg['physics'] = True
    edg['color'] = '#cf888f'

for n in nt.nodes:
    # tamanho dos nós
    n['shape'] = 'ellipse'
    size = 40*(cen_dict[n['label']]/max_cen)**0.4
    n['font'] = str(int(size))+'px arial white'
    n['color'] = '#7f333f'
    n['labelHighlightBold'] = True
    

def time_to_min(time):
    return str(time//60)+":"+"0"*(time%60<10)+str(time%60)

for n in nt.nodes:
    # links nos nós
    link_list = ["<a href='https://www.youtube.com/watch?v="+str(iden)+
                 "&t="+str(time)+"s' target='_blank' rel='noopener noreferrer'>"+time_to_min(time)+"<br>" for time in dict_moments[n['label']]]
    soma = ''
    for k in link_list:
        soma += k
    n['title'] = soma
    #print(n['size'])

    
nt.show("text_network.html")

In [181]:
#print(nx.adjacency_matrix(text_network))
#print(nt.nodes)
lista = []
for node in nt.nodes:
    lista.append(node['label'])
print(lista)

['spinning', 'power', 'women', 'even', 'time', 'people', 'working', 'life', 'industrial', 'work', 'many', 'factories', 'see', 'water', 'Revolution', 'could', 'one', 'Industrial', 'porcelain', 'make', 'like', 'machinery', 'workers', 'century', 'often']


In [174]:
SUS = banStopword(lista)
print(SUS)

0.6883577704429626 ('doing', 0.6883577704429626) working
0.7955270409584045 ('most', 0.7955270409584045) many
0.6570900082588196 ('get', 0.6570900082588196) see
0.8800161480903625 ('can', 0.8800161480903625) could
0.7335886359214783 ('another', 0.7335886359214783) one
(['working', 'many', 'see', 'could', 'one'], [('doing', 0.6883577704429626), ('most', 0.7955270409584045), ('get', 0.6570900082588196), ('can', 0.8800161480903625), ('another', 0.7335886359214783)])
